# Image Data Processing with PyTorch: Data Loaders and Augmentation

In this tutorial, we will explore how to work with data loaders and apply data augmentation techniques using PyTorch. We will use the Intel Image Classification dataset. This dataset contains images of different landforms such as buildings, forest, glacier, mountain, sea, and street.

## Data Loading and Exploration
First, we will import the necessary libraries and load the Intel Image Classification dataset.

In [ ]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import ImageFolder
import os
import numpy as np
import pandas as pd

In [ ]:
# Define the path to the dataset
data_dir = '/kaggle/input/intel-image-classification'

# Define transformations for the dataset
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the dataset
dataset = ImageFolder(root=os.path.join(data_dir, 'seg_train/seg_train'), transform=transform)

In [ ]:
# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Display the number of images in the dataset
print(f"Number of training images: {train_size}")
print(f"Number of validation images: {val_size}")

In [ ]:
# Display some sample images from the dataset
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# Get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))

## Data Augmentation Techniques
Data augmentation is crucial for training robust models. We will apply various augmentation techniques to our dataset.

In [ ]:
# Define augmented transformations
augmented_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomResizedCrop((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#### Random Horizontal Flip:

* **Description**: This transformation randomly flips the image horizontally with a probability of 0.5. This means that each image has a 50% chance of being flipped.

* **Purpose**: It helps in making the model invariant to the horizontal orientation of the objects in the image, thereby increasing the robustness of the model.

#### Random Rotation:

* **Description**: This transformation rotates the image by a random angle selected from a range of -10 to 10 degrees.

* **Purpose**: It helps the model to learn features irrespective of the rotational orientation of the objects, making the model more robust to rotations.

#### Color Jitter:

* **Description**: This transformation randomly changes the brightness, contrast, saturation, and hue of the image.
> * Brightness: Adjusts the brightness of the image.
> * Contrast: Adjusts the contrast of the image.
> * Saturation: Adjusts the color saturation of the image.
> Hue: Adjusts the hue (color) of the image.

* **Purpose**: It introduces variations in lighting and color conditions, helping the model to generalize better to different environments and lighting conditions.

#### Random Resized Crop:

* **Description**: This transformation first randomly crops a part of the image and then resizes it to a specified size (128x128 in this case).

* **Purpose**: It helps the model to focus on different parts of the image, thereby learning more diverse features.

In [ ]:
# Apply the augmented transformations to the training dataset
augmented_train_dataset = ImageFolder(root=os.path.join(data_dir, 'seg_train/seg_train'), transform=augmented_transform)

# Create a data loader for the augmented dataset
augmented_train_loader = DataLoader(augmented_train_dataset, batch_size=32, shuffle=True)

## Creating Custom Data Loaders
We will create custom data loaders to handle the original and augmented datasets efficiently.

In [ ]:
# Function to create data loaders
def create_data_loaders(data_dir, batch_size=32, augmented=False):
    if augmented:
        transform = augmented_transform
    else:
        transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    
    dataset = ImageFolder(root=data_dir, transform=transform)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

In [ ]:
# Create data loaders for the original and augmented datasets
train_loader, val_loader = create_data_loaders(os.path.join(data_dir, 'seg_train/seg_train'), augmented=False)
augmented_train_loader, _ = create_data_loaders(os.path.join(data_dir, 'seg_train/seg_train'), augmented=True)

## Visualizing Augmented Data
To better understand the effects of data augmentation, we will visualize some augmented images.

In [ ]:
# Function to visualize augmented images
def visualize_augmented_data(data_loader):
    dataiter = iter(data_loader)
    images, labels = next(dataiter)
    
    # Show images
    imshow(torchvision.utils.make_grid(images))
    # Print labels
    print(' '.join('%5s' % labels[j].item() for j in range(4)))

# Visualize augmented training data
visualize_augmented_data(augmented_train_loader)
